# :::::::::::::::::::::::::::::::::  🤩  WELCOME  🤩  :::::::::::::::::::::::::::::::::

A company has customer data that contains 8 columns of customer details and another table having name customer_policy data contains the policy details of the customer. 

The company intends to offer some discount in premium for certain customers. To do that they ask their Data scientist team to get some information. Hence, following tasks DS team decided to perform:

### 1. Add the column names to both datasets:

#### i. Column Name for customer details table:

>            customer_id, 
           Gender,
           age, 
           driving licence present,
           region code, 
           previously insured, 
           vehicle age 
           and vehicle damage, in respective order. 

#### ii. Column Name for customer_policy table:

>            customer_id, 
           annual premium (in Rs), 
           sales channel code, 
           vintage,
           response. 

In [2]:
# Import the files 
import pandas as pd
import numpy as np

# Load customer details dataset
customer_details = pd.read_csv("customer_details.csv")

# Load customer policy details dataset
customer_policy = pd.read_csv("customer_policy_details.csv")

# add column names to the customer_details data frame 
customer_details.columns=["customer_id",'Gender','Age','Driving_License','Region_Code','Previously_Insured',
                          'Vehicle_Age','Vehicle_Damage']
# Set customer_id column as index
customer_details.set_index('customer_id', inplace=True)

# add column names to the customer_policy data frame
customer_policy.columns=['customer_id','Annual_Premium','Policy_Sales_Channel','Vintage','Response']
# Set customer_id column as index
customer_policy.set_index('customer_id', inplace=True)


In [3]:
customer_details.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage
customer_id,,,,,,,
1.0,Male,44.0,1.0,28.0,0.0,> 2 Years,Yes
2.0,Male,76.0,1.0,3.0,0.0,1-2 Year,No
3.0,Male,47.0,1.0,28.0,0.0,> 2 Years,Yes
4.0,Male,21.0,1.0,11.0,1.0,< 1 Year,No
5.0,Female,29.0,1.0,41.0,1.0,< 1 Year,No


In [4]:
customer_policy.head()

,Annual_Premium,Policy_Sales_Channel,Vintage,Response
customer_id,,,,
1.0,40454.0,26.0,217.0,1.0
2.0,33536.0,26.0,183.0,0.0
3.0,38294.0,26.0,27.0,1.0
4.0,28619.0,152.0,203.0,0.0
5.0,27496.0,152.0,39.0,0.0


### 2. Checking and Cleaning Data Quality:











#### i. Null values

Generate a summary of count of all the null values column wise
Drop Null values for customer_id because central tendencies for id’s is not feasible.
Replace all null values for numeric columns by mean. 
Replace all null values for Categorical value by mode.

Here is the code to generate the summary:

In [5]:
# Check for null values in customer_details dataframe
print(customer_details.isnull().sum())

# Check for null values in customer_policy dataframe
print(customer_policy.isnull().sum())


Gender                368
Age                   368
Driving_License       393
Region_Code           392
Previously_Insured    381
Vehicle_Age           381
Vehicle_Damage        407
dtype: int64
Annual_Premium          346
Policy_Sales_Channel    400
Vintage                 388
Response                361
dtype: int64


Next, to drop null values for customer_id, we can use the dropna() function in Pandas:

In [13]:
# Drop null values for all columns except customer_id
customer_details.dropna(subset=customer_details.columns[1:], inplace=True)
customer_policy.dropna(subset=customer_policy.columns[1:], inplace=True)


To replace null values for numeric columns by mean, we can use the fillna() function in Pandas:

In [14]:
# Replace all null values for numeric columns by mean
customer_details.fillna(customer_details.mean(numeric_only=True), inplace=True)

# Replace all null values for categorical columns by mode
customer_details = customer_details.apply(lambda x: x.fillna(x.mode()[0]))

# Replace null values for numeric columns by mean
customer_policy.fillna(customer_policy.mean(), inplace=True)

In [15]:
# Check for null values in customer_details dataframe
print(customer_details.isnull().sum())

# Check for null values in customer_policy dataframe
print(customer_policy.isnull().sum())

Gender                0
Age                   0
Driving_License       0
Region_Code           0
Previously_Insured    0
Vehicle_Age           0
Vehicle_Damage        0
dtype: int64
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64


Finally, to replace null values for categorical values by mode, we can use the fillna() function in Pandas:

In [16]:
# Replace null values for categorical values by mode
customer_details = customer_details.apply(lambda x:x.fillna(x.value_counts().index[0]))
customer_policy = customer_policy.apply(lambda x:x.fillna(x.value_counts().index[0]))


#### ii. Outliers

Generate a summary of count of all the outliers column wise
Replace all outlier values for numeric columns by mean. 
(Hint1: for outlier treatment use IQR method as follows:

For example: for a column X calculate Q1 = 25th percentile and Q3 = 75th percentile then IQR = Q3 – Q1 ) then to check outlier, anything lower than a Q1 – 1.5IQR or greater than Q3 + 1.5 IQR would be an outlier

Hint2: For getting percentile value, explore pd.describe() function)

In [36]:
import numpy as np

# Find outliers and replace with mean
for col in customer_details.select_dtypes(include=np.number).columns:
    Q1 = customer_details[col].describe()['25%']
    Q3 = customer_details[col].describe()['75%']
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    outlier_count = len(customer_details[(customer_details[col] < lower_bound) | (customer_details[col] > upper_bound)])
    print(f"Column {col} has {outlier_count} outliers.")
    customer_details[col] = np.where((customer_details[col] < lower_bound) | (customer_details[col] > upper_bound), customer_details[col].mean(), customer_details[col])
    

Column Age has 0 outliers.
Column Driving_License has 917 outliers.
Column Region_Code has 0 outliers.
Column Previously_Insured has 0 outliers.
Column female has 0 outliers.
Column male has 0 outliers.


#### iii. White spaces

Remove white spaces

In [19]:
customer_details = customer_details.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
customer_policy = customer_policy.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


#### iv. case correction(lower or upper, any one) 

In [20]:
customer_details['Gender'] = customer_details['Gender'].str.lower()


In [21]:
customer_details.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage
customer_id,,,,,,,
1.0,male,44.0,1.0,28.0,0.0,> 2 Years,Yes
2.0,male,76.0,1.0,3.0,0.0,1-2 Year,No
3.0,male,47.0,1.0,28.0,0.0,> 2 Years,Yes
4.0,male,21.0,1.0,11.0,1.0,< 1 Year,No
5.0,female,29.0,1.0,41.0,1.0,< 1 Year,No


#### v. Convert nominal data (categorical) into dummies 

for future modeling use if required

In [22]:
dummies = pd.get_dummies(customer_details['Gender'])
customer_details = pd.concat([customer_details, dummies], axis=1)


In [23]:
customer_details.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,female,male
customer_id,,,,,,,,,
1.0,male,44.0,1.0,28.0,0.0,> 2 Years,Yes,0,1
2.0,male,76.0,1.0,3.0,0.0,1-2 Year,No,0,1
3.0,male,47.0,1.0,28.0,0.0,> 2 Years,Yes,0,1
4.0,male,21.0,1.0,11.0,1.0,< 1 Year,No,0,1
5.0,female,29.0,1.0,41.0,1.0,< 1 Year,No,1,0


#### vi. Drop Duplicates (duplicated rows)

In [24]:
customer_details.drop_duplicates(inplace=True)
customer_policy.drop_duplicates(inplace=True)


In [25]:
customer_details.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,female,male
customer_id,,,,,,,,,
1.0,male,44.0,1.0,28.0,0.0,> 2 Years,Yes,0,1
2.0,male,76.0,1.0,3.0,0.0,1-2 Year,No,0,1
3.0,male,47.0,1.0,28.0,0.0,> 2 Years,Yes,0,1
4.0,male,21.0,1.0,11.0,1.0,< 1 Year,No,0,1
5.0,female,29.0,1.0,41.0,1.0,< 1 Year,No,1,0


### 3. Create a Master table for future use. Join the customer table and customer_policy table to get a master table using customer_id in both tables.

(Hint: use pd.merge() function)

In [26]:

master_table = pd.merge(customer_details, customer_policy, on='customer_id')


In [27]:
master_table.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,female,male,Annual_Premium,Policy_Sales_Channel,Vintage,Response
customer_id,,,,,,,,,,,,,
1.0,male,44.0,1.0,28.0,0.0,> 2 Years,Yes,0,1,40454.0,26.0,217.0,1.0
2.0,male,76.0,1.0,3.0,0.0,1-2 Year,No,0,1,33536.0,26.0,183.0,0.0
3.0,male,47.0,1.0,28.0,0.0,> 2 Years,Yes,0,1,38294.0,26.0,27.0,1.0
4.0,male,21.0,1.0,11.0,1.0,< 1 Year,No,0,1,28619.0,152.0,203.0,0.0
5.0,female,29.0,1.0,41.0,1.0,< 1 Year,No,1,0,27496.0,152.0,39.0,0.0


### 4. Company needs some important information from the master table to make decisions for future growth.They needs following information:

#### i. Gender wise average annual premium

In [28]:
gender_avg_premium = master_table.groupby('Gender')['Annual_Premium'].mean()
print(gender_avg_premium)


Gender
female    31428.397458
male      32531.925040
Name: Annual_Premium, dtype: float64


#### ii. Age wise average annual premium


In [29]:
age_avg_premium = master_table.groupby('Age')['Annual_Premium'].mean()
print(age_avg_premium)


Age
20.0    31686.266512
21.0    29002.078775
22.0    30445.759740
23.0    30203.965021
24.0    32338.763314
            ...     
81.0    36054.080000
82.0    38753.500000
83.0    37347.416667
84.0    43314.125000
85.0    38397.285714
Name: Annual_Premium, Length: 67, dtype: float64



#### iii. Is your data balanced between the genders?

          (Hint: Data is balanced if number of counts in each group is approximately same)

In [30]:
gender_counts = customer_details['Gender'].value_counts()
print(gender_counts)


male      14219
female    11946
Name: Gender, dtype: int64


In [31]:
gender_percentages = gender_counts / len(customer_details) * 100
print(gender_percentages)



male      54.343589
female    45.656411
Name: Gender, dtype: float64


#### iv. Vehicle age wise average annual premium.

In [32]:
master_table.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,female,male,Annual_Premium,Policy_Sales_Channel,Vintage,Response
customer_id,,,,,,,,,,,,,
1.0,male,44.0,1.0,28.0,0.0,> 2 Years,Yes,0,1,40454.0,26.0,217.0,1.0
2.0,male,76.0,1.0,3.0,0.0,1-2 Year,No,0,1,33536.0,26.0,183.0,0.0
3.0,male,47.0,1.0,28.0,0.0,> 2 Years,Yes,0,1,38294.0,26.0,27.0,1.0
4.0,male,21.0,1.0,11.0,1.0,< 1 Year,No,0,1,28619.0,152.0,203.0,0.0
5.0,female,29.0,1.0,41.0,1.0,< 1 Year,No,1,0,27496.0,152.0,39.0,0.0


In [33]:
vehicle_age_avg_premium = master_table.groupby('Vehicle_Age')['Annual_Premium'].mean()
print(vehicle_age_avg_premium)


Vehicle_Age
1-2 Year     31873.274383
< 1 Year     31672.376059
> 2 Years    34487.168970
Name: Annual_Premium, dtype: float64


### 5. Is there any relation between Person Age and annual premium?

Hint: use correlation function (Correlation describes the relationship between two variables). 

Correlation coefficient < -0.5           - Strong negative relationship

Correlation coefficient > 0.5            -  Strong positive relationship

-0.5 < Correlation coefficient < 0.5   - There is no relationship. 

In [34]:
correlation = master_table['Age'].corr(master_table['Annual_Premium'])
print("Relation between Person Age and annual premium: ",correlation)
print("-0.5 < Correlation coefficient < 0.5 - There is no relationship.")


Relation between Person Age and annual premium:  0.07073913643116346
-0.5 < Correlation coefficient < 0.5 - There is no relationship.


#  ::::::::::::::::::::::::::::::::: 🥳  THANK YOU 🥳  :::::::::::::::::::::::::::::::::
